# Supplementary Note 1. Mathematical model of regulatory RNA arrays

Below we describe a model that capture regulatory RNA arrays. First, we will describe the chemical reactions that describe the interaction of the Csy4 protein with the regulatory RNA arrays.



```
# This is formatted as code
```

### Chemical reactions
First, we describe the production and degradation of Csy4 as $P^*$ (unbound state), and $n$ copies of STAR in a single transcription as $X_n$.
\begin{align}
\emptyset \xrightarrow[]{\alpha} P^* && P^*  \xrightarrow{\delta} \emptyset \\
\emptyset \xrightarrow{\theta} X_n && X_n  \xrightarrow{\phi} \emptyset 
\end{align}

Next, we describe the binding between $P^*$ and $X_n$ as $P$ (bound state). Also, we describe the production of $n$ copies of activator $X$, and its degradation.
\begin{align}
X_n + nP^* \xrightarrow[]{k^+_P} P  && P  \xrightarrow{k^-_P} X_n + nP^* \\
P \xrightarrow[]{r_P} nX + && X  \xrightarrow{\phi} \emptyset \\
P \xrightarrow[]{\delta} \emptyset
\end{align}

Finally, we describe the interaction of the STAR $X$ and its target RNA.
\begin{align}
X + C^* \xrightarrow[]{k^+_C} C  && C  \xrightarrow{k^-_C} X + C^* \\
C \xrightarrow[]{r_C} Y && Y  \xrightarrow{\delta} \emptyset \\
C^* \xrightarrow[]{r_0} Y &&  C \xrightarrow[]{\xi} C^*
\end{align}

Using mass of action, we ca derive the Ordinary Differential Equations.

Cleaving of regulatory RNA arrays
\begin{align}
  \dot p^* &= \alpha - \delta p^* - nk^+_Px_np^* + nk^-_Pp\\
  \dot x_n &= \theta - \phi x_n -  k^+_Px_np^* + k^-_Pp \\
  \dot p   &= k^+_Px_np^* - k^-_Pp - r_P p - \delta p
\end{align}

Regulation of the cleaved STARs
\begin{align}
  \dot x &= n r_P p - \phi x - k^+_Cxc^* + k^-_Cc\\
  \dot c &= k^+_Cxc^* - k^-_Cc - r_C c - \xi c\\
  \dot y &= r_0 c^* + r_C c - \delta y 
\end{align}

In [1]:
import os, sys, subprocess
if "google.colab" in sys.modules:
  cmd = "pip install --upgrade biocircuits bokeh-catplot watermark blackcellmagic intersect"
  process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  stdout, stderr = process.communicate()
# ------
import numpy as np
import scipy.integrate
import scipy.optimize

import bokeh.io
import bokeh.plotting
from bokeh.io import export_svgs

from bokeh.models import LinearColorMapper, ColorBar
from bokeh.models import Range1d
from bokeh.models import LinearAxis

import seaborn as sns
import matplotlib.pyplot as plt

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
# Writing function for ODE model
def model_rhs(state, t, a, th, ph, d, kp1,kn1, kp2, kn2, r0,r1, r2, n, ct, xi):
  """
  Right hand desribe production and degradation and return dx/dt
  """
  ps, xn, p, x, c, y = state
  cs = ct-c
  return (
      [ 
          a - d*ps - n*kp1*xn*ps + n*kn1*p, # d y1 / dt = 
          th - ph*xn - kp1*xn*ps + kn1*p,
          kp1*xn*ps - kn1*p - r1*p - d*p,
          n*r1*p - ph*x - kp2*x*cs + kn2*c,
          kp2*x*cs - kn2*c - r2*c - xi*c,
          r0*cs + r2*c - d*y,
      ]
  )


In [3]:
a = 1
th = 1
ph = 1
d = 1
kp1 = 100
kn1 = 1
kp2 = 100
kn2 = 1
r0 = 0.1
r1 = 10
r2 = 10
n = 1
ct = 1
xi = 0

args = (a, th, ph, d, kp1,kn1, kp2, kn2, r0,r1, r2, n, ct, xi)

# Solving ODE
t = np.linspace(0,10,400)

x01 = np.array([0., 0., 0., 0., 0., 0]) # condiciones iniciales
# different solvesrs solve_ivp(), ode(), and odeint()
x = scipy.integrate.odeint(model_rhs, x01, t, args=args)
x1 = x.transpose()


# Set up plots
size = [300, 200] # 300/200, 144/110
plots = [
    bokeh.plotting.figure(
        plot_width=size[0],plot_height=size[1],),
    bokeh.plotting.figure(
        plot_width=size[0],plot_height=size[1],),
]

# Graficando y1 en el tiempo para dos condiciones inicials diferentes
plots[0].line(t,x1[3,:],line_width=3, color="orange",) # mRNA (m) en el tiempo
plots[1].line(t,x1[5,:],line_width=3, color="orange",) # mRNA (m) en el tiempo

bokeh.io.show(bokeh.layouts.row(plots))

In [4]:
a = 1
th = 1
ph = 1
d = 1
kp1 = 100
kn1 = 1
kp2 = 100
kn2 = 1
r0 = 0.1
r1 = 10
r2 = 10
n = 1
ct = 1
xi = 0
args = (a, th, ph, d, kp1,kn1, kp2, kn2, r0,r1, r2, n, ct, xi)

# Solving ODE
t = np.linspace(0,10,400)
x01 = np.array([0., 0., 0., 0., 0., 0]) # condiciones iniciales

N = 50
vec = np.linspace(0,1,N)
y_v = vec*0
for i in range(N):
  args = (a, vec[i], ph, d, kp1,kn1, kp2, kn2, r0,r1, r2, n, ct, xi)
  # different solvesrs solve_ivp(), ode(), and odeint()
  x = scipy.integrate.odeint(model_rhs, x01, t, args=args)
  x1 = x.transpose()
  y_v[i] = x1[5,-1]


# Set up plots
fig_size = [150, 115] # for paper size (144, 115) # for larger size (144,144)
fig_size = [360, 275] # to visualize
x_range = (0, 1)
y_range = (0, 1)

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          #x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"



plots[0].line(vec,y_v, line_width=2, color="black")

#plots[0].output_backend = "svg"

bokeh.io.show(bokeh.layouts.row(plots))

In [5]:
a = 1
th = 1
ph = 1
d = 1
kp1 = 100
kn1 = 1
kp2 = 100
kn2 = 1
r0 = 0.
r1 = 10
r2 = 10
n = 2
ct = 1
xi = 0

args = (a, th, ph, d, kp1,kn1, kp2, kn2, r0,r1, r2, n, ct, xi)

# Solving ODE
t = np.linspace(0,10,400)
x01 = np.array([0., 0., 0., 0., 0., 0]) # condiciones iniciales

N1 = 5
vec1 = np.logspace(-1,1,N1)

N2 = 30
vec2 = np.linspace(0,1,N2)

palette = bokeh.palettes.Blues7[::-1]

# Set up plots
fig_size = [150, 115] # for paper size (144, 115) # for larger size (144,144)
fig_size = [360, 275] # to visualize
x_range = (0, 1)
y_range = (0, 1)

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"

for i in range(N1):
  y_v = vec2*0
  for j in range(N2):
    args = (a, vec2[j], ph, d, kp1*vec1[i],kn1, kp2, kn2, r0,r1, r2, n, ct, xi)
    # different solvesrs solve_ivp(), ode(), and odeint()
    x = scipy.integrate.odeint(model_rhs, x01, t, args=args)
    x1 = x.transpose()
    y_v[j] = x1[5,-1]
  
  plots[0].line(vec2,y_v, line_width=2, color=palette[i+2])

#plots[0].output_backend = "svg"

bokeh.io.show(bokeh.layouts.row(plots))

# Quantifying cleaving efficiency
We use the binding and unbinding rates as a way to tune the efficiency of cleaving. By tunning the dissociation rate, we can reduce the complex and therefore decrease the efficiency of cleaving.

In [6]:
a = 2 #3
th = 1
ph = 1
d = 1
kp1 = 100
kn1 = 1
kp2 = 100
kn2 = 1
r0 = 0.1 # 0.1/5
r1 = 10
r2 = 10
n = 1
ct = 1
xi = 0

args = (a, th, ph, d, kp1,kn1, kp2, kn2, r0,r1, r2, n, ct, xi)

# Solving ODE
t = np.linspace(0,10,400)
x01 = np.array([0., 0., 0., 0., 0., 0]) # condiciones iniciales

N1 = 4
vec1 = np.linspace(1,N1,N1)
vec1 = np.array([1,2,4,8])
N2 = 50
vec2 = np.linspace(0,1,N2)

N3 = 3 #6
vec3 = np.linspace(0,1,N3)[1:-1]
 
palette2 = bokeh.palettes.Blues7[::-1]
#palette1 = bokeh.palettes.Greys7[::-1]
palette1 = bokeh.palettes.Oranges7[::-1]

# Set up plots
fig_size = [150, 115] # for paper size (144, 115) # for larger size (144,144)
fig_size = [160, 125] # for paper size (144, 115) # for larger size (144,144)
#fig_size = [360, 275] # to visualize
x_range = (0, 1)
y_range = (0, 1)

plots = []
for i in range(2):
    plots.append(bokeh.plotting.figure(tools="save",plot_width=fig_size[0], plot_height=fig_size[1],
                          #x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "8pt"     #background_fill_color="#fafafa"

    
for i in range(N1):
    y_v1 = vec2*0
    y_v2 = vec2*0
    for j in range(N2):
        args = (a, vec2[j], ph, d, kp1,kn1, kp2, kn2, 0,r1, r2, vec1[i], ct, xi)
        # different solvesrs solve_ivp(), ode(), and odeint()
        x = scipy.integrate.odeint(model_rhs, x01, t, args=args)
        x1 = x.transpose()
        y_v1[j] = x1[5,-1]

        args = (a, vec2[j], ph, d, kp1,kn1, kp2, kn2, 0,r1/4, r2, vec1[i], ct, xi)
        # different solvesrs solve_ivp(), ode(), and odeint()
        x = scipy.integrate.odeint(model_rhs, x01, t, args=args)
        x1 = x.transpose()
        y_v2[j] = x1[5,-1]
        
    plots[0].line(vec2,y_v1, line_width=2, color=palette1[i+2],line_dash="dashed")
    plots[0].line(vec2,y_v2, line_width=2, color=palette1[i+2])

plots[0].line((0.5,0.5),(0,2), line_width=1, color="black")

for j in range(N3-2):
    y_v1 = vec1*0
    y_v2 = vec1*0
    for i in range(N1):
        args = (a, vec3[j], ph, d, kp1,kn1, kp2, kn2, 0,r1, r2, vec1[i], ct, xi)
        # different solvesrs solve_ivp(), ode(), and odeint()
        x = scipy.integrate.odeint(model_rhs, x01, t, args=args)
        x1 = x.transpose()
        y_v1[i] = x1[5,-1]

        args = (a, vec3[j], ph, d, kp1,kn1, kp2, kn2, 0,r1/4, r2, vec1[i], ct, xi)
        # different solvesrs solve_ivp(), ode(), and odeint()
        x = scipy.integrate.odeint(model_rhs, x01, t, args=args)
        x1 = x.transpose()
        y_v2[i] = x1[5,-1]
    
    plots[1].line(vec1,y_v1, line_width=2, color="grey",line_dash="dashed")
    plots[1].circle(vec1,y_v1, line_width=2, color="grey",size = 7, fill_color ="white")
    
    plots[1].line(vec1,y_v2, line_width=2, color="grey")
    plots[1].circle(vec1,y_v2, line_width=2, color="grey",size = 7)
        
        
plots[0].x_range = Range1d(0,1)
plots[0].y_range = Range1d(0,2)

plots[1].x_range = Range1d(0.8,4.2)
plots[1].y_range = Range1d(0,2)

plots[0].output_backend = "svg"
plots[1].output_backend = "svg"

bokeh.io.show(bokeh.layouts.row(plots))

# Quantifying leaky expression
 

In [25]:
a = 2 #3
th = 1
ph = 1
d = 1
kp1 = 100
kn1 = 1
kp2 = 100
kn2 = 1
r0 = 1 # 0.1/5
r1 = 10
r2 = 10
n = 1
ct = 1
xi = 0

args = (a, th, ph, d, kp1,kn1, kp2, kn2, r0,r1, r2, n, ct, xi)

# Solving ODE
t = np.linspace(0,10,400)
x01 = np.array([0., 0., 0., 0., 0., 0]) # condiciones iniciales

N1 = 4
vec1 = np.linspace(1,N1,N1)

N2 = 50
vec2 = np.linspace(0,1,N2)

N3 = 3
vec3 = np.linspace(0,1,N3)[1:-1]
 
palette2 = bokeh.palettes.Blues7[::-1]
#palette1 = bokeh.palettes.Greys7[::-1]
palette1 = bokeh.palettes.Oranges7[::-1]

# Set up plots
fig_size = [150, 115] # for paper size (144, 115) # for larger size (144,144)
fig_size = [160, 125] # for paper size (144, 115) # for larger size (144,144)
#fig_size = [360, 275] # to visualize
x_range = (0, 1)
y_range = (0, 1)

plots = []
for i in range(2):
    plots.append(bokeh.plotting.figure(tools="save",plot_width=fig_size[0], plot_height=fig_size[1],
                          #x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "8pt"     #background_fill_color="#fafafa"

for i in range(N1):
    y_v1 = vec2*0
    y_v2 = vec2*0
    for j in range(N2):
        args = (a, vec2[j], ph, d, kp1,kn1, kp2, kn2, 0,r1, r2, vec1[i], ct, 0)
        # different solvesrs solve_ivp(), ode(), and odeint()
        x = scipy.integrate.odeint(model_rhs, x01, t, args=args)
        x1 = x.transpose()
        y_v1[j] = x1[5,-1]

        args = (a, vec2[j], ph, d, kp1,kn1, kp2, kn2, r0,r1, r2, vec1[i], ct, 0)
        # different solvesrs solve_ivp(), ode(), and odeint()
        x = scipy.integrate.odeint(model_rhs, x01, t, args=args)
        x1 = x.transpose()
        y_v2[j] = x1[5,-1]
        
    plots[0].line(vec2,y_v1, line_width=2, color=palette2[i+2],line_dash="dashed")
    plots[0].line(vec2,y_v2, line_width=2, color=palette2[i+2])

plots[0].line((0.5,0.5),(0,3), line_width=1, color="black")

for j in range(N3-2):
    y_v1 = vec1*0
    y_v2 = vec1*0
    for i in range(N1):
        args = (a, vec3[j], ph, d, kp1,kn1, kp2, kn2, 0,r1, r2, vec1[i], ct, 0)
        # different solvesrs solve_ivp(), ode(), and odeint()
        x = scipy.integrate.odeint(model_rhs, x01, t, args=args)
        x1 = x.transpose()
        y_v1[i] = x1[5,-1]

        args = (a, vec3[j], ph, d, kp1,kn1, kp2, kn2, r0,r1, r2, vec1[i], ct, 0)
        # different solvesrs solve_ivp(), ode(), and odeint()
        x = scipy.integrate.odeint(model_rhs, x01, t, args=args)
        x1 = x.transpose()
        y_v2[i] = x1[5,-1]
    
    plots[1].line(vec1,y_v1, line_width=2, color="grey",line_dash="dashed")
    plots[1].circle(vec1,y_v1, line_width=2, color="grey",size = 7, fill_color ="white")
    
    plots[1].line(vec1,y_v2, line_width=2, color="grey")
    plots[1].circle(vec1,y_v2, line_width=2, color="grey",size = 7)
        
        
plots[0].x_range = Range1d(0,1)
plots[0].y_range = Range1d(0,3)

plots[1].x_range = Range1d(0.8,4.2)
plots[1].y_range = Range1d(0,3)

plots[0].output_backend = "svg"
plots[1].output_backend = "svg"

bokeh.io.show(bokeh.layouts.row(plots))